In [1]:
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage

#create mol2
smi = '[NH3+][C@@H](C/C(C)=C/N(C)C)C([O-])=O'
!obabel -:"{smi} uaa" -O "./uaa.mol2" --gen3d

#rdkit editable mol2 mol
mol = Chem.MolFromMol2File('./uaa.mol2')
Chem.MolToPDBFile(mol, './uaa.pdb')
mol = Chem.MolFromPDBFile('./uaa.pdb')
mol = Chem.AddHs(mol) #H omitted in pdb

smarts = Chem.MolFromSmarts("[OX1-]-[CX3](=[OX1])-[CX4]-[NX4+]") #assume protonated, single OC(=O)(N)C
anames = ['OXT', 'C', 'O', 'CA', 'N']
mat = mol.GetSubstructMatches(smarts)[0]

for i, idx in enumerate(mat): #label amino acid skeleton atoms
    atom = mol.GetAtomWithIdx(idx)
    atom.GetPDBResidueInfo().SetName(' ' + anames[i] + ' '*(3-len(anames[i])))

h_idx = 1
pdbi = atom.GetPDBResidueInfo() #copy of residue info
#last atom is N
#amino acid skeleton amine-H
for neighbor in atom.GetNeighbors():
    if neighbor.GetAtomicNum() != 1:
        continue
    name = f'H{h_idx}'
    pdbi.SetName(' ' + name + ' '*(3-len(name)) )
    h_idx += 1
    neighbor.SetPDBResidueInfo(pdbi)
    
elem_idxs = {}
for atom in mol.GetAtoms():
    if not atom.GetPDBResidueInfo(): #non amino acid N skeleton Hs...
        name = f'H{h_idx}'
        pdbi.SetName(' ' + name + ' '*(3-len(name)) )
        h_idx += 1
        atom.SetPDBResidueInfo(pdbi)
    else:
        if atom.GetIdx() in mat:
            continue
        elem = ''.join([c for c in atom.GetPDBResidueInfo().GetName() if not c.isdigit() and c != ' ']) #get only char
        if not elem_idxs.get(f'{elem}'): elem_idxs[f'{elem}'] = 0
        elem_idxs[f'{elem}'] += 1
        elem_idx = elem_idxs[f'{elem}']
        name = f'{elem}{elem_idxs[f"{elem}"]}'
        atom.GetPDBResidueInfo().SetName(' ' + name + ' '*(3-len(name)))

Chem.MolToPDBFile(mol, "./new_uaa.pdb")
cmd = f'obabel -ipdb uaa.pdb -omol2 -O new_uaa.mol2 --title uaa --gen3d'
!{cmd}

1 molecule converted
1 molecule converted
